In [0]:
use catalog identifier(:catalog);
use schema identifier(:schema);

In [0]:
declare or replace qry_str string;

set var qry_str = 
  "create or replace view metv_usage
  (
    Date comment 'Date of the usage record, this field can be used for faster aggregation by date.',
    `Account ID` comment 'The ID of the Databricks account.',
    `Workspace Name` comment 'The name of the workspace.',
    `Warehouse Name` comment 'The name of the SQL warehouse.',
    `Usage Unit` comment 'Unit this usage is measured in. Possible values include DBUs.',
    `Usage Type` comment 'The type of usage attributed to the product or workload for billing purposes. Possible values are COMPUTE_TIME, STORAGE_SPACE, NETWORK_BYTES, API_CALLS, TOKEN, or GPU_TIME.',
    `Billing Origin Product` comment 'The product that originated the usage. Some products can be billed as different SKUs. For possible values, see [View information about the product associated with the usage](https://docs.databricks.com/en/admin/system-tables/billing.html#features).',
    `SKU Name` comment 'Name of the SKU',
    `Usage Sum` comment 'Number of units consumed for this record.'
  )
  with metrics
  language yaml
  as $$
  version: 0.1
  
  source: " || :catalog || '.' || :schema || ".fct_usage
  
  joins:
    - name: calendar
      source: " || :catalog || '.' || :schema || ".dim_calendar
      on: calendar.calendar_key = source.calendar_key

    - name: compute
      source: " || :catalog || '.' || :schema || ".dim_compute
      on: compute.compute_key = source.compute_key

    - name: workspace
      source: " || :catalog || '.' || :schema || ".dim_workspace
      on: workspace.workspace_key = source.workspace_key
  
  dimensions:
    - name: Date
      expr: calendar.calendar_date
    - name: Account ID
      expr: workspace.account_id
    - name: Workspace Name
      expr: workspace.workspace_name
    - name: Warehouse Name
      expr: compute.warehouse_name
    - name: Usage Unit
      expr: usage_unit
    - name: Usage Type
      expr: usage_type
    - name: Billing Origin Product
      expr: billing_origin_product
    - name: SKU Name
      expr: sku_name
  
  measures:
    - name: Usage Sum
      expr: SUM(usage_quantity)
  $$";

execute immediate qry_str;